# KPI Calculation


In [11]:
#### Data preperation
import pandas as pd
import numpy as np


df_boston = pd.read_csv('./data/boston_2015.csv', dtype={'start_station_id': np.int64, 'end_station_id': 'string', 'end_station_name': 'string', 'start_station_name': 'string', 'bike_id': np.int64, 'user_type': 'string'})
df_boston['end_station_id'].unique()
df_boston.drop(index=df_boston.loc[df_boston["end_station_id"] == "\\N"].index, inplace=True, axis=1)
df_boston = df_boston.astype({'end_station_id': np.int64})
df_boston['start_time'] = pd.to_datetime(df_boston['start_time'], format='%Y-%m-%d %X')
df_boston['end_time'] = pd.to_datetime(df_boston['end_time'], format='%Y-%m-%d %X')
len(df_boston)

1122557

### Utilization
#### Usage Rate (travel time / total possible travel time)

In [78]:
from datetime import datetime, timedelta
amountOfBikes = len(df_boston['bike_id'].unique())

possibleTravelTime = amountOfBikes*365*24*60
possibleTravelTime = timedelta(minutes=possibleTravelTime)

df_boston['trip_length'] = (df_boston['end_time'] - df_boston['start_time'])
realTravelTime = df_boston['trip_length'].sum()

usageRate = realTravelTime.total_seconds() / possibleTravelTime.total_seconds()
print(usageRate*100)

2.8636253630430217


In [81]:
df_usage_rate = df_boston.copy()
df_usage_rate.dtypes
# df_usage_rate['Weekday'] = df_usage_rate['start_time'].dt.day_name()
# df_usage_rate.loc[df_usage_rate['start_time'] < datetime(2015,1,2)].head()
# df_usage_rate['start_hour'] = df_usage_rate['start_time'].dt.hour
# df_usage_rate['end_hour'] = df_usage_rate['end_time'].dt.hour
# df_usage_rate_overlapping = df_usage_rate.loc[df_usage_rate['start_hour'] != df_usage_rate['end_hour']]
df_usage_rate['times_tmp'] = df_usage_rate.apply((lambda x: [x[0].round('60min') - timedelta(seconds=1), x[0].round('60min')] if x[0].hour != x[0].hour else None), axis=0)
# df_usage_rate.apply(lambda x: print(x.start_time), axis=1)
# df_usage_rate.set_index('start_time')
# df_usage_rate[df_usage_rate['start_time']=='2015-01-01 01:28:27']



#df_rentals_month_weekday= df_usage_rate.groupby(['Month', 'Weekday']).size().reset_index(name='num_rides')

#df_rentals_month_weekday.Weekday = pd.Categorical(df_rentals_month_weekday.Weekday,categories=days)
#df_rentals_month_weekday = df_rentals_month_weekday.sort_values('Weekday')

#df_rentals_month_weekday.Month = pd.Categorical(df_rentals_month_weekday.Month,categories=months)
#df_rentals_month_weekday = df_rentals_month_weekday.sort_values('Month')

#sns.lineplot(data=df_rentals_month_weekday, x='Weekday', y='num_rides', hue='Month')

start_time             datetime64[ns]
end_time               datetime64[ns]
start_station_id                int64
end_station_id                  int64
start_station_name             string
end_station_name               string
bike_id                         int64
user_type                      string
trip_length           timedelta64[ns]
dtype: object

In [76]:
df_usage_rate.iloc[0][1]

Timestamp('2015-01-01 00:30:47')

In [52]:
df_usage_rate['start_time'][8].round('60min') - timedelta(seconds=1)

Timestamp('2015-01-01 04:59:59')

### Coverage

### Revenue

### Clustering